<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-&amp;-Prep" data-toc-modified-id="Imports-&amp;-Prep-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports &amp; Prep</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Make-a-DF" data-toc-modified-id="Make-a-DF-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Make a DF</a></span></li><li><span><a href="#Roster-DF" data-toc-modified-id="Roster-DF-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Roster DF</a></span></li><li><span><a href="#Get-Stats" data-toc-modified-id="Get-Stats-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Get Stats</a></span></li><li><span><a href="#League-Leaders" data-toc-modified-id="League-Leaders-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>League Leaders</a></span></li></ul></div>

# Imports & Prep

In [8]:
import requests
import time
import pandas as pd
# import mlbstatsapi
import statsapi

# Functions

In [9]:
def get_teams():
    """Fetch all MLB teams."""
    try:
        response = requests.get("https://statsapi.mlb.com/api/v1/teams", 
                                params={"sportId": 1})
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(F"Error!: {e}")
        return{}

def make_teams_df(teams_json):
    if teams_json and 'teams' in teams_json:
        teams_df = pd.json_normalize(teams_json['teams'])
    else:
        print("No Teams JSON file.")
    return(teams_df)
    
    
    
def get_roster(team_id):
    """ Get roster for a given team"""
    try:
        url = f"https://statsapi.mlb.com/api/v1/teams/{team_id}/roster"
        response = requests.get(url)
        response.raise_for_status()
        roster = response.json()
        return roster['roster']
    except requests.RequestException as e:
        print(f"Error in roster request for {team_id}: {e}")
        
def get_team_stats(team_id):
    """ Get stats for a team"""
    try:
        url = f"https://statsapi.mlb.com/api/v1/teams/{team_id}/stats"
        response = requests.get(url)
        response.raise_for_status()
        team_stats = response.json()
        return team_stats
    except requests.RequestException as e:
        print(f"Error getting team stats for team {team_id}: {e}")
        return {}

def get_player_stats(player_id,season = 2024):
    try:
        url = f"https://statsapi.mlb.com/api/v1/people/{player_id}/stats"
        response = requests.get(url)
        response.raise_for_status()
        player_stats = response.json()
        return player_stats
    except requests.RequestException as e:
        print(f"Error gettingstats for player {player_id}: {e}")
        return {}
        
def all_player_stats(player_id, group="[hitting,pitching,fielding]", type_var="season", sportId=1):
    result = statsapi.player_stat_data(player_id, group, type_var, sportId)
    

# Make a DF

In [10]:
teams = get_teams()
teams_df = make_teams_df(teams)

In [11]:
team_id_list = teams_df['id'].to_list()

# Roster DF

In [12]:
all_players = []

for team in team_id_list:
    roster = get_roster(team)
    team_name = teams_df.loc[teams_df['id']== team, 'name'].iloc[0] 
    for player in roster:
        player_data = {
            'person_iD': player['person']['id'],
            'name': player['person']['fullName'],
            'number': player['jerseyNumber'],
            'team_name': team_name,
            'team_id' : player['parentTeamId'],
            'position' : player['position']['name'],
            'pos_abv' : player['position']['abbreviation'],
            'status' : player['status']['description']    
        }
        all_players.append(player_data)
        
meta_roster = pd.DataFrame(all_players)
meta_roster

,person_iD,name,number,team_name,team_id,position,pos_abv,status
0,605156,Aaron Brooks,53,Oakland Athletics,133,Pitcher,P,Active
1,647351,Abraham Toro,31,Oakland Athletics,133,Third Base,3B,Active
2,649557,Aledmys Díaz,6,Oakland Athletics,133,Shortstop,SS,Active
3,613534,Austin Adams,29,Oakland Athletics,133,Pitcher,P,Active
4,667670,Brent Rooker,25,Oakland Athletics,133,Outfielder,LF,Active
...,...,...,...,...,...,...,...,...
777,656887,Sean Reid-Foley,71,New York Mets,121,Pitcher,P,Active
778,516782,Starling Marte,6,New York Mets,121,Outfielder,RF,Active
779,621512,Tomás Nido,3,New York Mets,121,Catcher,C,Active
780,656731,Tylor Megill,38,New York Mets,121,Pitcher,P,Active


# Get Stats

In [30]:
ty_france = statsapi.player_stat_data(664034, group="[hitting,fielding]", type="season", sportId=1)

{'id': 664034, 'first_name': 'Ty', 'last_name': 'France', 'active': True, 'current_team': 'Seattle Mariners', 'position': '1B', 'nickname': None, 'last_played': None, 'mlb_debut': '2019-04-26', 'bat_side': 'Right', 'pitch_hand': 'Right', 'stats': [{'type': 'season', 'group': 'fielding', 'season': '2024', 'stats': {'gamesPlayed': 52, 'gamesStarted': 50, 'assists': 42, 'putOuts': 324, 'errors': 2, 'chances': 368, 'fielding': '.995', 'position': {'code': '3', 'name': 'First Base', 'type': 'Infielder', 'abbreviation': '1B'}, 'rangeFactorPerGame': '7.04', 'rangeFactorPer9Inn': '7.45', 'innings': '442.0', 'games': 52, 'doublePlays': 30, 'triplePlays': 0, 'throwingErrors': 0}}, {'type': 'season', 'group': 'fielding', 'season': '2024', 'stats': {'gamesPlayed': 1, 'gamesStarted': 0, 'assists': 1, 'putOuts': 1, 'errors': 0, 'chances': 2, 'fielding': '1.000', 'position': {'code': '4', 'name': 'Second Base', 'type': 'Infielder', 'abbreviation': '2B'}, 'rangeFactorPerGame': '2.00', 'rangeFactorPer9

In [45]:
hit_list = []
pitch_list = []
field_list = []

player_stats = pd.DataFrame()

for index, player in meta_roster.iterrows():
    i_id = player['person_iD']
    stats = statsapi.player_stat_data(i_id, group="[hitting,pitching,fielding]", type="season", sportId=1)
    
    for stat in stats['stats']:
        if stat['group'] == 'pitching':
            pitching_stats = stat['stats']
        
            pitching_data = {
                'person_id': stats['id'],
                'games_played': pitching_stats.get('gamesPlayed', 0),
                'starts': pitching_stats.get('gamesStarted', 0),
                'ground_outs': pitching_stats.get('groundOuts', 0),
                'air_outs': pitching_stats.get('airOuts', 0),
                'runs': pitching_stats.get('runs', 0),
                'doubles': pitching_stats.get('doubles', 0),
                'triples': pitching_stats.get('triples', 0),
                'home_runs': pitching_stats.get('homeRuns', 0),
                'strike_outs': pitching_stats.get('strikeOuts', 0),
                'base_on_balls': pitching_stats.get('baseOnBalls', 0),
                'intentional_walks': pitching_stats.get('intentionalWalks', 0),
                'hits': pitching_stats.get('hits', 0),
                'hit_batters': pitching_stats.get('hitByPitch', 0),
                'average': pitching_stats.get('avg', '.000'),
                'at_bats': pitching_stats.get('atBats', 0),
                'obp': pitching_stats.get('obp', '.000'),
                'slg': pitching_stats.get('slg', '.000'),
                'ops': pitching_stats.get('ops', '.000'),
                'caught_stealing': pitching_stats.get('caughtStealing', 0),
                'stolen_bases': pitching_stats.get('stolenBases', 0),
                'stolen_base_percentage': pitching_stats.get('stolenBasePercentage', '.000'),
                'ground_into_double_play': pitching_stats.get('groundIntoDoublePlay', 0),
                'number_of_pitches': pitching_stats.get('numberOfPitches', 0),
                'era': pitching_stats.get('era', '0.00'),
                'innings_pitched': pitching_stats.get('inningsPitched', '0.0'),
                'wins': pitching_stats.get('wins', 0),
                'losses': pitching_stats.get('losses', 0),
                'saves': pitching_stats.get('saves', 0),
                'save_opportunities': pitching_stats.get('saveOpportunities', 0),
                'holds': pitching_stats.get('holds', 0),
                'blown_saves': pitching_stats.get('blownSaves', 0),
                'earned_runs': pitching_stats.get('earnedRuns', 0),
                'whip': pitching_stats.get('whip', '0.00'),
                'batters_faced': pitching_stats.get('battersFaced', 0),
                'outs': pitching_stats.get('outs', 0),
                'games_pitched': pitching_stats.get('gamesPitched', 0),
                'complete_games': pitching_stats.get('completeGames', 0),
                'shutouts': pitching_stats.get('shutouts', 0),
                'strikes': pitching_stats.get('strikes', 0),
                'strike_percentage': pitching_stats.get('strikePercentage', '.000'),
                'hit_batsmen': pitching_stats.get('hitBatsmen', 0),
                'balks': pitching_stats.get('balks', 0),
                'wild_pitches': pitching_stats.get('wildPitches', 0),
                'pickoffs': pitching_stats.get('pickoffs', 0),
                'total_bases': pitching_stats.get('totalBases', 0),
                'ground_outs_to_air_outs': pitching_stats.get('groundOutsToAirouts', '.000'),
                'win_percentage': pitching_stats.get('winPercentage', '.000'),
                'pitches_per_inning': pitching_stats.get('pitchesPerInning', '0.00'),
                'games_finished': pitching_stats.get('gamesFinished', 0),
                'strikeout_walk_ratio': pitching_stats.get('strikeoutWalkRatio', '0.00'),
                'strikeouts_per_9_inn': pitching_stats.get('strikeoutsPer9Inn', '0.00'),
                'walks_per_9_inn': pitching_stats.get('walksPer9Inn', '0')
            }

            pitch_list.append(pitching_data)
        if stat['group'] == 'hitting':
            hitting_stats = stat['stats']
            hitting_data = {
                'person_id': stats['id'],
                'games_played': hitting_stats.get('gamesPlayed', 0),
                'ground_outs': hitting_stats.get('groundOuts', 0),
                'air_outs': hitting_stats.get('airOuts', 0),
                'runs': hitting_stats.get('runs', 0),
                'doubles': hitting_stats.get('doubles', 0),
                'triples': hitting_stats.get('triples', 0),
                'home_runs': hitting_stats.get('homeRuns', 0),
                'strike_outs': hitting_stats.get('strikeOuts', 0),
                'base_on_balls': hitting_stats.get('baseOnBalls', 0),
                'intentional_walks': hitting_stats.get('intentionalWalks', 0),
                'hits': hitting_stats.get('hits', 0),
                'hit_by_pitch': hitting_stats.get('hitByPitch', 0),
                'avg': hitting_stats.get('avg', '.000'),
                'at_bats': hitting_stats.get('atBats', 0),
                'obp': hitting_stats.get('obp', '.000'),
                'slg': hitting_stats.get('slg', '.000'),
                'ops': hitting_stats.get('ops', '.000'),
                'caught_stealing': hitting_stats.get('caughtStealing', 0),
                'stolen_bases': hitting_stats.get('stolenBases', 0),
                'stolen_base_percentage': hitting_stats.get('stolenBasePercentage', '.000'),
                'ground_into_double_play': hitting_stats.get('groundIntoDoublePlay', 0),
                'total_bases': hitting_stats.get('totalBases', 0),
                'rbi': hitting_stats.get('rbi', 0),
                'left_on_base': hitting_stats.get('leftOnBase', 0),
                'sac_bunts': hitting_stats.get('sacBunts', 0),
                'sac_flies': hitting_stats.get('sacFlies', 0),
                'babip': hitting_stats.get('babip', '.000')
            }
            hit_list.append(hitting_data)

        if stat['group'] == 'fielding':
            fielding_stats = stat['stats']
            fielding_data = {
                'person_id': stats['id'],
                'games_played': fielding_stats.get('gamesPlayed', 0),
                'games_started': fielding_stats.get('gamesStarted', 0),
                'assists': fielding_stats.get('assists', 0),
                'put_outs': fielding_stats.get('putOuts', 0),
                'errors': fielding_stats.get('errors', 0),
                'chances': fielding_stats.get('chances', 0),
                'fielding_percentage': fielding_stats.get('fielding', '.000'),
                'range_factor_per_game': fielding_stats.get('rangeFactorPerGame', '0.00'),
                'range_factor_per_9_inn': fielding_stats.get('rangeFactorPer9Inn', '0.00'),
                'innings_played': fielding_stats.get('innings', '0.0'),
                'double_plays': fielding_stats.get('doublePlays', 0),
                'triple_plays': fielding_stats.get('triplePlays', 0),
                'position_code': fielding_stats['position']['code'],
                'position_name': fielding_stats['position']['name'],
                'position_type': fielding_stats['position']['type'],
                'position_abbreviation': fielding_stats['position']['abbreviation'],
                'throwing_errors': fielding_stats.get('throwingErrors', 0)
            }
            field_list.append(fielding_data)
            


        

    
mlb_hitting = pd.DataFrame(hit_list)
mlb_pitching = pd.DataFrame(pitch_list)
mlb_fielding = pd.DataFrame(field_list)
  
    
  
    
    

In [46]:
mlb_pitching

,person_id,games_played,starts,ground_outs,air_outs,runs,doubles,triples,home_runs,strike_outs,...,wild_pitches,pickoffs,total_bases,ground_outs_to_air_outs,win_percentage,pitches_per_inning,games_finished,strikeout_walk_ratio,strikeouts_per_9_inn,walks_per_9_inn
0,605156,3,3,19,23,11,2,0,3,9,...,0,0,33,0.83,.000,14.94,0,1.80,4.67,2.60
1,613534,25,0,14,11,6,2,0,1,25,...,2,0,19,1.27,.---,20.02,3,3.13,12.98,4.15
2,676664,11,11,51,85,27,12,3,8,40,...,0,0,92,0.60,.571,15.73,0,2.35,5.97,2.54
3,672552,1,0,3,4,0,0,0,0,2,...,0,0,2,0.75,.---,14.00,1,2.00,6.00,3.00
4,683155,4,4,8,33,14,9,1,2,20,...,0,0,36,0.24,.500,15.97,0,3.33,8.71,2.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,622663,10,10,66,57,26,6,1,4,50,...,1,1,62,1.16,.500,16.28,0,1.92,7.67,3.99
420,657585,20,0,15,21,12,3,0,2,43,...,2,1,33,0.71,.714,17.01,4,3.31,14.16,4.28
421,640455,10,10,35,67,20,10,0,3,47,...,0,1,63,0.52,.750,17.90,0,2.14,8.24,3.86
422,656887,15,0,14,8,8,2,0,0,20,...,1,0,12,1.75,.333,18.68,7,1.82,12.27,6.75


In [47]:
mlb_fielding

,person_id,games_played,games_started,assists,put_outs,errors,chances,fielding_percentage,range_factor_per_game,range_factor_per_9_inn,innings_played,double_plays,triple_plays,position_code,position_name,position_type,position_abbreviation,throwing_errors
0,605156,3,3,1,2,0,3,1.000,1.00,1.59,17.1,0,0,1,Pitcher,Pitcher,P,0
1,647351,3,3,2,10,0,12,1.000,4.00,5.40,20.0,3,0,3,First Base,Infielder,1B,0
2,647351,16,14,30,20,0,50,1.000,3.13,3.69,122.0,8,0,4,Second Base,Infielder,2B,0
3,647351,21,20,35,14,1,50,.980,2.33,2.64,167.0,2,0,5,Third Base,Infielder,3B,0
4,647351,1,1,0,2,0,2,1.000,2.00,2.25,8.0,0,0,7,Outfielder,Outfielder,LF,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1369,656731,3,3,2,0,0,2,1.000,0.67,1.13,16.0,1,0,1,Pitcher,Pitcher,P,0
1370,621438,12,7,1,16,1,18,.944,1.42,2.07,74.1,0,0,7,Outfielder,Outfielder,LF,1
1371,621438,10,9,0,30,0,30,1.000,3.00,3.75,72.0,0,0,8,Outfielder,Outfielder,CF,0
1372,621438,11,7,1,16,0,17,1.000,1.55,2.01,76.0,0,0,9,Outfielder,Outfielder,RF,0


In [48]:
mlb_hitting

,person_id,games_played,ground_outs,air_outs,runs,doubles,triples,home_runs,strike_outs,base_on_balls,...,caught_stealing,stolen_bases,stolen_base_percentage,ground_into_double_play,total_bases,rbi,left_on_base,sac_bunts,sac_flies,babip
0,647351,54,45,61,28,12,1,5,38,9,...,2,2,.500,2,86,19,69,0,0,.329
1,649557,1,0,0,0,0,0,0,1,0,...,0,0,.---,0,0,0,0,0,0,.---
2,667670,44,18,37,23,10,0,11,59,20,...,0,2,1.000,0,88,34,58,0,0,.382
3,663662,3,3,2,2,0,0,1,2,0,...,0,0,.---,1,6,2,10,0,0,.286
4,605204,33,40,18,11,4,0,4,28,5,...,0,0,.---,3,41,5,50,0,0,.266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,553882,27,17,25,4,2,0,0,12,3,...,0,0,.---,2,12,5,25,1,0,.196
439,624413,55,57,60,29,11,0,12,47,20,...,0,1,1.000,8,96,26,97,0,1,.240
440,516782,51,59,38,27,7,0,6,46,12,...,0,8,1.000,3,75,19,76,1,1,.314
441,621512,28,19,22,7,1,0,3,18,3,...,0,0,.---,0,27,7,30,2,1,.264
